In [1]:
import pandas as pd
import numpy as np
import string
import keras
import tensorflow
import nltk
nltk.download('all')
import threading # will potentially use multi-threading
from nltk.stem.porter import *
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

Using TensorFlow backend.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package cess_cat 

[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downloading package universal_treebanks_v20 to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package universal_treebanks_v20 is already up-to-
[nltk_data]    |       date!
[nltk_data]    | Downloading package verbnet to
[nltk_data]    |     /Users/haritha/nltk_data...
[nltk_data]    |   Package verbnet is already up-to-date!
[nltk_data]    | Downloading package verbnet3 to
[nltk_data]    |     /Users/haritha/nltk_data...

In [2]:
from nltk.corpus import words
import os
from sklearn.manifold import TSNE
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
df_review = pd.read_csv("df_review.csv")
df_review = df_review[['id','reviews.text', 'reviews.rating']]
df_review = df_review.dropna(subset = [ 'reviews.rating'])
df_review.head()

,id,reviews.text,reviews.rating
0,AVpe7AsMilAPnD_xQ78G,I initially had trouble deciding between the p...,5.0
1,AVpe7AsMilAPnD_xQ78G,Allow me to preface this with a little history...,5.0
2,AVpe7AsMilAPnD_xQ78G,I am enjoying it so far. Great for reading. Ha...,4.0
3,AVpe7AsMilAPnD_xQ78G,I bought one of the first Paperwhites and have...,5.0
4,AVpe7AsMilAPnD_xQ78G,I have to say upfront - I don't like coroporat...,5.0


In [4]:
# Clean String
def cleanString(review,stopWords):
    """
    Cleans input string using set rules.
    Cleaning rules:         Every word is lemmatized and lowercased. Stopwords and non alpha-numeric words are removed.
                            Each sentence ends with a period.
    Input:   review       - string(in sentence structure)
             stopWords    - set of strings which should be removed from review
    Output:  returnString - cleaned input string
             idx_list     - list of lists, one list is equal to one sentence. In every list are the index
                            of each word as they appeared in the non cleaned sentence
                            e.g. nonCleaned = "This is a test." -> cleaned = "This test." -> cleaned_index = [[0,3]]
    """
    # Init the Wordnet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    returnString = ""
    sentence_token = sent_tokenize(review)
    idx_list = []
    for j in range(len(sentence_token)):
        single_sentence = word_tokenize(sentence_token[j])
        sentences_filtered = [(idx,lemmatizer.lemmatize(w.lower())) for idx,w in enumerate(single_sentence)
                              if w.lower() not in stopwords.words('english') and w.isalnum()]
        idx_list.append([x[0] for x in sentences_filtered])
        word_list = [x[1] for x in sentences_filtered]
        returnString = returnString + ' '.join(word_list) + ' . '
    
    return returnString, idx_list, single_sentence


In [5]:
# Parameters
MAX_SENTENCE_NUM = 9
MAX_WORD_NUM = 40
MAX_FEATURES = 200000 

# Tokenization
# Word index

n = df_review.shape[0]

"""
Using the keras Tokenizer class a word index is built.
The most 'MAX_FEATURES' used words are tokenized to a number.
this dictionary is saved in word_index
"""
texts = []
for i in range(n):
    s = df_review['reviews.text'].iloc[i]
    s = ' '.join([word.strip(string.punctuation) for word in s.split() if word.strip(string.punctuation) is not ""])
    texts.append(s)
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_FEATURES,lower=True, oov_token=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index

print('Number of tokens: ' + str(len(word_index)))

Number of tokens: 5757


In [6]:
df_review['reviews.text'][890]

"I habe the echo and I bought this one fort my son's room but it's not as user friendly. I wish I would've just bought the other one."

In [7]:

cleanString(df_review['reviews.text'][890], stopwords)

('habe echo bought one fort son room user friendly . wish would bought one . ',
 [[1, 3, 6, 8, 9, 11, 13, 19, 20], [1, 3, 6, 9]],
 ['I',
  'wish',
  'I',
  'would',
  "'ve",
  'just',
  'bought',
  'the',
  'other',
  'one',
  '.'])

In [8]:
# Load word vectors from pre-trained dataset
embeddings_index = {}
f = open(os.path.join(os.getcwd(), 'glove.6B.100d.txt'), encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))



Found 400000 word vectors.


In [9]:
# Embedding

EMBED_SIZE = 100

min_wordCount = 2
absent_words = 0
small_words = 0
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
word_counts = tokenizer.word_counts
for word, i in word_index.items():
    if word_counts[word] > min_wordCount:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
        else:
            absent_words += 1
    else:
        small_words += 1
print('Total absent words are', absent_words, 'which is', "%0.2f" % (absent_words * 100 / len(word_index)),
      '% of total words')
print('Words with '+str(min_wordCount)+' or less mentions', small_words, 'which is', "%0.2f" % (small_words * 100 / len(word_index)),
      '% of total words')
print(str(len(word_index)-small_words-absent_words) + ' words to proceed.')

Total absent words are 138 which is 2.40 % of total words
Words with 2 or less mentions 2792 which is 48.50 % of total words
2827 words to proceed.


In [10]:
embedding_matrix[word_index['wish']]

array([-0.18573999,  0.33739999,  0.22002   , -0.17411999, -0.20810001,
        0.065849  , -0.67724001, -0.021098  ,  0.34239   , -0.28331   ,
       -0.24129   ,  0.32093999,  0.63810003, -0.20578   , -0.49623001,
       -0.38701001,  0.26163   ,  0.57085001, -1.04390001,  0.84697998,
       -0.32833001,  0.093014  , -0.66772002, -0.34823   , -0.20953   ,
        0.41198   ,  0.1163    , -0.25376999,  1.03299999, -0.50541002,
       -0.13981   ,  0.85241002,  0.45214999,  0.18548   ,  0.18926001,
        0.65697998,  0.32034001,  0.065963  ,  0.10379   , -0.66953999,
       -0.34965   ,  0.40617999,  0.61378002, -0.69779003, -0.41830999,
       -0.17193   ,  0.057787  , -0.11909   , -0.033902  , -1.04929996,
        0.25382999,  0.19976   ,  0.13756999,  0.38247001,  0.34845999,
       -1.38569999,  0.22776   , -0.041017  ,  0.95434999, -0.085851  ,
       -0.23647   ,  0.72874999, -0.76482999, -0.34062999,  0.61935002,
        0.26714   , -0.05441   ,  0.62133998, -0.46303999, -0.82

In [26]:
def split_df(dataframe, column_name, training_split, validation_split, test_split):
    """
    Splits a pandas dataframe into trainingset, validationset and testset in specified ratio.
    All sets are balanced, which means they have the same ratio for each category as the full set.
    Input:   dataframe        - Pandas Dataframe, should include a column for data and one for categories
             column_name      - Name of dataframe column which contains the categorical output values
             training_split   - from ]0,1[, default = 0.6
             validation_split - from ]0,1[, default = 0.2        
             test_split       - from ]0,1[, default = 0.2
                                Sum of all splits need to be 1
    Output:  train            - Pandas DataFrame of trainset
             validation       - Pandas DataFrame of validationset
             test             - Pandas DataFrame of testset
    """
    if training_split + validation_split + test_split != 1.0:
        raise ValueError('Split paramter sum should be 1.0')
        
    total = len(dataframe.index)
 
    train = dataframe.reset_index().groupby(column_name).apply(lambda x: x.sample(frac=training_split))\
    .reset_index(drop=True).set_index('index')
    train = train.sample(frac=1)
    temp_df = dataframe.drop(train.index)
    validation = temp_df.reset_index().groupby(column_name)\
    .apply(lambda x: x.sample(frac=validation_split/(test_split+validation_split)))\
           .reset_index(drop=True).set_index('index')
    validation = validation.sample(frac=1)
    test = temp_df.drop(validation.index)
    test = test.sample(frac=1)
    
    print('Total: ', len(dataframe))
    print('Training: ', len(train), ', Percentage: ', len(train)/len(dataframe))
    print('Validation: ', len(validation), ', Percentage: ', len(validation)/len(dataframe))
    print('Test:', len(test), ', Percentage: ', len(test)/len(dataframe))

    return train, validation, test

train, validation, test = split_df(df_review['reviews.text'], 'reviews.text',0.75,0.15,0.1)


Total:  1177
Training:  1103 , Percentage:  0.9371282922684792
Validation:  50 , Percentage:  0.04248088360237893
Test: 24 , Percentage:  0.020390824129141887


In [24]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K

class AttentionLayer(Layer):
    """
    Hierarchial Attention Layer as described by Hierarchical Attention Networks for Document Classification(2016)
    - Yang et. al.
    Source: https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf
    Theano backend
    """
    def __init__(self,attention_dim=100,return_coefficients=False,**kwargs):
        # Initializer 
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        # casting "Dimension" type of input_shape's elements to "int" type
        #input_shape = tuple([i if (isinstance(i, float) or i is None else i.value for i in input_shape)]) 
        self.W = K.variable(self.init((input_shape[-1].value, self.attention_dim)),name='W')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self._trainable_weights = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

In [33]:
"""
Create Keras functional model for hierarchical attention network
"""
import tensorflow.keras
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Bidirectional, GRU, Dense
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import initializers
from tensorflow.python.keras.layers import TimeDistributed
from tensorflow.python.keras.layers import Dropout
embedding_layer = Embedding(len(word_index) + 1,EMBED_SIZE,weights=[embedding_matrix], 
                            input_length=MAX_WORD_NUM, trainable=False,name='word_embedding')

# Words level attention model
word_input = Input(shape=(MAX_WORD_NUM,), dtype='int32',name='word_input')
word_sequences = embedding_layer(word_input)
word_gru = Bidirectional(GRU(50, return_sequences=True),name='word_gru')(word_sequences)
word_dense = Dense(100, activation='relu', name='word_dense')(word_gru) 
word_att,word_coeffs = AttentionLayer(EMBED_SIZE,True,name='word_attention')(word_dense)
wordEncoder = Model(inputs = word_input,outputs = word_att)


# Sentence level attention model
sent_input = Input(shape=(MAX_SENTENCE_NUM,MAX_WORD_NUM), dtype='int32',name='sent_input')
print("sent_input ",sent_input)
sent_encoder = TimeDistributed(wordEncoder,name='sent_linking')(sent_input)
sent_gru = Bidirectional(GRU(50, return_sequences=True),name='sent_gru')(sent_encoder)
sent_dense = Dense(100, activation='relu', name='sent_dense')(sent_gru) 
sent_att,sent_coeffs = AttentionLayer(EMBED_SIZE,return_coefficients=True,name='sent_attention')(sent_dense)
sent_drop = Dropout(0.5,name='sent_dropout')(sent_att)
preds = Dense(5, activation='softmax',name='output')(sent_drop)

# Model compile
model = Model(sent_input, preds)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
print(wordEncoder)
print("-------------------")
print(model.summary())

sent_input  Tensor("sent_input_7:0", shape=(?, 9, 40), dtype=int32)
-------------------
Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sent_input (InputLayer)      [(None, 9, 40)]           0         
_________________________________________________________________
sent_linking (TimeDistribute (None, 9, 100)            641400    
_________________________________________________________________
sent_gru (Bidirectional)     (None, 9, 100)            45300     
_________________________________________________________________
sent_dense (Dense)           (None, 9, 100)            10100     
_________________________________________________________________
sent_attention (AttentionLay [(None, 100), (None, 9, 1 10200     
_________________________________________________________________
sent_dropout (Dropout)       (None, 100)               0         
____________________________________

Issues faced
1. float() argument must be a string or a number, not 'Dimension'-> self.W = K.variable(self.init((input_shape[-1]**.value**, self.attention_dim)),name='W')<br>
2. trainable weights read-only<br>
3. tuple cant be layer -> version error install tensorflow 1.14 and import all libs from tensorflow.python.keras


https://www.kaggle.com/hsankesara/news-classification-using-han/notebook

<tf.Tensor 'output_1/Softmax:0' shape=(?, 5) dtype=float32>